In [1]:
import numpy as np
import glob
import os
import pandas as pd
import pickle


In [2]:
pkl_file_path = "/home/jony/Git/GAN-N-Net/datasets/new_raw_data/site4_28_classes_ready2go.pkl"

In [ ]:
# pd.read_pickle(pkl_file_path)
import dill
import pandas as pd

def read_pickle_in_chunks(file_path, chunk_size=10):
    with open(file_path, 'rb') as f:
        while True:
            try:
                chunk = dill.load(f)
                yield pd.DataFrame(chunk)
            except EOFError:
                break

# Usage
file_path = 'large_pickle_file.pkl'
chunk_size = 1000

for i, chunk in enumerate(read_pickle_in_chunks(file_path, chunk_size)):
    print(f"Chunk {i+1}:")
    print(chunk.head())